In [ ]:
! pip install -U bitsandbytes
! pip install -U accelerate transformers

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSequenceClassification
from datasets import load_dataset
import torch

In [ ]:
model_name = "google/gemma-2b-it"
token = "YOUR TOKEN"
tokenizer = AutoTokenizer.from_pretrained(model_name, token =token)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    token="YOUR TOKEN"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
prompt = "tell  me a scary  story"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tell  me a scary  story about a young woman named Sarah who was stalked by a stalker named John.

**Sarah's Nightmare**

Sarah was a young woman with a vibrant spirit and a heart full of kindness. But her life took a sinister turn when she became the target of a relentless stalker named John. John's obsession with Sarah was as terrifying as it was captivating. He would stalk her through the streets, break into her apartment, and leave cryptic messages that sent chills down her spine.

One fateful night


In [ ]:
dataset = load_dataset("yahma/alpaca-cleaned", split="train[:1000]")

In [ ]:
type(dataset)

datasets.arrow_dataset.Dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head(5)

,output,input,instruction
0,1. Eat a balanced and nutritious diet: Make su...,,Give three tips for staying healthy.
1,"The three primary colors are red, blue, and ye...",,What are the three primary colors?
2,An atom is the basic building block of all mat...,,Describe the structure of an atom.
3,There are several ways to reduce air pollution...,,How can we reduce air pollution?
4,I had to make a difficult decision when I was ...,,Pretend you are a project manager of a constru...


In [ ]:
df.shape

(1000, 3)

In [ ]:
df['input'],
df['output'],

(0      1. Eat a balanced and nutritious diet: Make su...
 1      The three primary colors are red, blue, and ye...
 2      An atom is the basic building block of all mat...
 3      There are several ways to reduce air pollution...
 4      I had to make a difficult decision when I was ...
                              ...                        
 995    A stock split is a corporate action in which a...
 996    Japanese cuisine has a flavorful yet subtle ta...
 997    1. Confederation and Expansion (1867): In 1867...
 998    Individual performance refers to how well a si...
 999    The feasibility of opening a high-end gourmet ...
 Name: output, Length: 1000, dtype: object,)

In [ ]:
def format_prompt(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    return {"prompt": prompt}

dataset = dataset.map(format_prompt)

def tokenize(example):
    tokens = tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"]  # just for Gemma :))
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gemma-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipython-input-2923562311.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.437300
20,1.322200
30,1.199900
40,1.235800
50,1.195700
60,1.230900
70,1.224500
80,1.217700
90,1.314100
100,1.164400


TrainOutput(global_step=250, training_loss=1.2034682922363282, metrics={'train_runtime': 499.9147, 'train_samples_per_second': 2.0, 'train_steps_per_second': 0.5, 'total_flos': 6091180081152000.0, 'train_loss': 1.2034682922363282, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./gemma-lora-adapter")

In [ ]:
def chat(instruction, input_text=""):
    model.eval()
    device = model.device

    prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
chat("Translate the following sentence to French:", "I am very happy to see you.")

### Instruction:
Translate the following sentence to French:

### Input:
I am very happy to see you.

### Response:
Je suis très reconnaissant de voir-vous.

### Explanation:
The French sentence "Je suis très reconnaissant de voir-vous" means "I am very happy to see you". It is a polite expression of gratitude and is used to express appreciation or recognition to someone who has done something nice for you.


In [ ]:
chat("Write Python code that prints all even numbers from 1 to 20.")

### Instruction:
Write Python code that prints all even numbers from 1 to 20.

### Input:


### Response:
```python
for i in range(1, 21):
    if i % 2 == 0:
        print(i)
```

This code uses a for loop to iterate through the numbers from 1 to 20. For each number i in this range, the if statement checks if the remainder of i divided by 2 is equal to 0, meaning that i is even. If it is, the code prints the value of i.

Output:
```python
2
4
6
8
10
12
14
16
18



In [ ]:
for i in range(1, 21):
    if i % 2 == 0:
        print(i)

2
4
6
8
10
12
14
16
18
20


## bert

In [ ]:
dataset = load_dataset("imdb")
dataset = dataset.shuffle(seed=42)

In [ ]:
df2 = pd.DataFrame(dataset)

In [ ]:
train_dataset = dataset["train"].select(range(2000))
test_dataset = dataset["test"].select(range(500))

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
lora_config = LoraConfig(
    r=8,                  # rank
    lora_alpha=16,        # alpha scaling
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="./lora-distilbert-imdb",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
#     tokenizer=tokenizer
# )

/tmp/ipython-input-963760692.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
10,0.727100
20,0.705600
30,0.675300
40,0.687100
50,0.632900
60,0.647200
70,0.633300
80,0.582900
90,0.616100
100,0.459000


TrainOutput(global_step=500, training_loss=0.5007087564468384, metrics={'train_runtime': 52.7122, 'train_samples_per_second': 37.942, 'train_steps_per_second': 9.485, 'total_flos': 67369703424000.0, 'train_loss': 0.5007087564468384, 'epoch': 1.0})

In [ ]:
def predict(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()

    labels = {0: "Negative", 1: "Positive"}
    return labels[predicted_class]


In [ ]:
print(predict("This movie was amazing! I loved every second of it."))
# ➜ Positive

print(predict("This movie was a disaster. Total waste of time."))
# ➜ Negative

print(predict("it was bullshit."))
# ➜ Negative

print(predict("it was cool man."))
# ➜ Positive

print(predict("what was that."))
# ➜ Negative

print(predict("this is what we call a movie."))
# ➜ Positive

Positive
Negative
Negative
Positive
Negative
Positive
